1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2022-12-02 18:26:34--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2022-12-02 18:26:34--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc617d68eb34a594e71609b66094.dl.dropboxusercontent.com/cd/0/inline/Bx3aslJaUKlr0jOvJSbF8Tl4oNLrnBrIgGnQfYmno6SSya40cP5G6siG1YEkbygkAl3b4fC-JDHcZNhLMiSH5qm1bI6fAv27tu5QDD1jfM94kcwMzpaxW0aGFt5pEi7Bx0g2DqBkJNN6TkXJWs5cC1jm567-TVU6-T1SJ5xCEK4DCw/file# [following]
--2022-12-02 18:26:35--  https://uc617d68eb34a594e71609b66094.dl.dropboxusercontent.com/cd/0/inline/Bx3aslJaUKlr0jOvJSbF8Tl4oNLrnBrIgGnQfYmno6SSya40cP5G6siG1YEkbygkAl3b4fC-JDHcZNhLMiSH5qm1bI6fAv27t

  7850K .......... .......... .......... .......... .......... 24% 7.16M 6s
  7900K .......... .......... .......... .......... .......... 24% 5.96M 6s
  7950K .......... .......... .......... .......... .......... 24% 5.21M 6s
  8000K .......... .......... .......... .......... .......... 24% 4.02M 6s
  8050K .......... .......... .......... .......... .......... 24% 5.27M 6s
  8100K .......... .......... .......... .......... .......... 25% 4.13M 6s
  8150K .......... .......... .......... .......... .......... 25% 3.75M 6s
  8200K .......... .......... .......... .......... .......... 25% 4.75M 6s
  8250K .......... .......... .......... .......... .......... 25% 7.50M 6s
  8300K .......... .......... .......... .......... .......... 25% 3.93M 6s
  8350K .......... .......... .......... .......... .......... 25% 8.82M 6s
  8400K .......... .......... .......... .......... .......... 26% 4.06M 6s
  8450K .......... .......... .......... .......... .......... 26% 5.63M 6s
  8500K ....

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [5]:
import pandas as pd 
import numpy as np

In [6]:
df = pd.read_csv('./data/data_000637.txt') 
size= df.shape
Rows=size[0]
N=12000
# print(N)
# print(size)

rows=df.head(100)
Range=df.iloc[10001:Rows]
Range
    

FileNotFoundError: [Errno 2] No such file or directory: './data/data_000637.txt'

2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [ ]:
max_bx = max(df.BX_COUNTER)
max_bx = max_bx+1
print("The maximum value of BX-Counter is = ", max_bx)

3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [1]:
df['abs_NS'] = (df['ORBIT_CNT'] * max_bx * 25 + df['BX_COUNTER'] * 25 + df['TDC_MEAS'] * 25/30)- nano_sec_init
df

NameError: name 'df' is not defined

4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [ ]:
first_row = df.iloc[0]
print(first_row)

nano_sec_init = first_row['ORBIT_CNT'] * max_bx * 25 + first_row['BX_COUNTER'] * 25 + first_row['TDC_MEAS'] * 25/30
# print(nano_sec_init)
sec_init = nano_sec_init *10**-9

print(str(dt.timedelta(seconds=sec_init)))

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [ ]:
noisy_grp = df.groupby('TDC_CHANNEL').sum()
noisy_grp

In [ ]:
sort = noisy_grp.sort_values(by=['HEAD'],ascending=False)
# display(sort)

disp_3 = sort.iloc[:3]
display(disp_3)

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [ ]:
non_empty = df.groupby('ORBIT_CNT').size().reset_index(name='counts')
l=len(non_empty)
print("Non-Empty Orbit ",l,"\n",non_empty)

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [ ]:
unique_orb = df[df['TDC_CHANNEL']==139].groupby('ORBIT_CNT').size().reset_index(name='counts')
ul=len(unique_orb)
print("Unique values",ul,"\n",unique_orb)

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [ ]:
tdc_0 = df.groupby('TDC_CHANNEL').size().reset_index(name='counts')
tdc_1 = df[df['FPGA']== 1].groupby('TDC_CHANNEL').size().reset_index(name='COUNT')


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.